<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2023-03-13 20:08:40--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  9.70MB/s    in 20s     

2023-03-13 20:09:01 (12.7 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [1]:
unzip concrete_data_week4.zip

In [3]:
!rm -r __MACOSX

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [13]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.models import load_model

In [5]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [6]:
data_generator = ImageDataGenerator(
    preprocessing_function=vgg16_preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'data/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

val_generator = data_generator.flow_from_directory(
    'data/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')



Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [ ]:
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False
print(model.layers)
model.summary()

[<keras.engine.functional.Functional object at 0x7fe5803beb80>, <keras.layers.core.dense.Dense object at 0x7fe58042abb0>]
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_2 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(val_generator)
num_epochs = 2

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-19-1e79dc354f6d>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model.fit_generator(


Epoch 1/2
301/301 [==============================] - 169s 558ms/step - loss: 0.1325 - accuracy: 0.9602 - val_loss: 0.0361 - val_accuracy: 0.9926
Epoch 2/2
301/301 [==============================] - 168s 557ms/step - loss: 0.0268 - accuracy: 0.9940 - val_loss: 0.0212 - val_accuracy: 0.9949


In [ ]:
model.save('classifier_vgg16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
!cp /content/gdrive/MyDrive/classifier_resnet_model.h5 .
!cp /content/gdrive/MyDrive/classifier_vgg16_model.h5 .

In [9]:
model_resnet50 = load_model('classifier_resnet_model.h5')
model = load_model('classifier_vgg16_model.h5')

In [10]:
data_generator_resnet = ImageDataGenerator(
    preprocessing_function=resnet_preprocess_input
)

test_generator_resnet = data_generator_resnet.flow_from_directory(
    'data/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical', shuffle=False)

test_generator_vgg16 = data_generator.flow_from_directory(
    'data/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical', shuffle=False)

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [ ]:
scores = model.evaluate_generator(test_generator_vgg16, len(test_generator_vgg16))
print('VGG16: Test Loss = {}, Test Accuracy = {}'.format(scores[0], scores[1]))

<ipython-input-42-0ae825e57552>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator_vgg16, len(test_generator_vgg16))


VGG16: Test Loss = 0.01806773990392685, Test Accuracy = 0.9959999918937683


In [ ]:
scores = model_resnet50.evaluate_generator(test_generator_resnet, len(test_generator_resnet))
print('Resnet50: Test Loss = {}, Test Accuracy = {}'.format(scores[0], scores[1]))

<ipython-input-45-a63ed30530b9>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model_resnet50.evaluate_generator(test_generator_resnet, len(test_generator_resnet))


Resnet50: Test Loss = 0.0022436468861997128, Test Accuracy = 1.0


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [21]:
predictions_vgg16 = (np.argmax(model.predict_generator(test_generator_vgg16, verbose=0), axis=1))
print('Model VGG16 Predictions of first 5 test images.')
for i in range(5):
  if predictions_vgg16[i] == 0:
    print('Image {}, Prediction: Negative'.format(i+1))
  else:
    print('Image {}, Prediction: Positive'.format(i+1))

<ipython-input-21-2f572db94d38>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_vgg16 = (np.argmax(model.predict_generator(test_generator_vgg16, verbose=0), axis=1))


Model VGG16 Predictions of first 5 test images.
Image 1, Prediction: Negative
Image 2, Prediction: Negative
Image 3, Prediction: Negative
Image 4, Prediction: Negative
Image 5, Prediction: Negative


In [22]:
predictions_resnet = np.argmax(model_resnet50.predict_generator(test_generator_resnet, verbose=0), axis=1)
print('Model RESNET50 Predictions of first 5 test images.')
for i in range(5):
  if predictions_resnet[i] == 0:
    print('Image {}, Prediction: Negative'.format(i+1))
  else:
    print('Image {}, Prediction: Positive'.format(i+1))

<ipython-input-22-7c0e021f2af8>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions_resnet = np.argmax(model_resnet50.predict_generator(test_generator_resnet, verbose=0), axis=1)


Model RESNET50 Predictions of first 5 test images.
Image 1, Prediction: Negative
Image 2, Prediction: Negative
Image 3, Prediction: Negative
Image 4, Prediction: Negative
Image 5, Prediction: Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).